In [124]:
import pandas as pd
import nltk
import csv

from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
#from googletrans import Translator

stemmer = SnowballStemmer('english')
#translator = Translator()
from pathlib import Path
import math
'''
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)
'''

from IPython.display import HTML, display


In [60]:
#nltk.download()

In [61]:
# Reading tsvs and getting cleaned stemmed dictionary

In [62]:
#load the given dataset into pandas for processing using only the needed columns
data = pd.read_csv('Airbnb_Texas_Rentals.csv', nrows=10, usecols= ['average_rate_per_night', 'bedrooms_count','city', 'date_of_listing', 'description', 'latitude','longitude','title', 'url'])


In [63]:
#create a file for every row in data(the given data of airbnb)
fileCount = 0
for index, r in data.iterrows():
    data_temp = data.loc[index:index]
    fileCount +=1
    data_temp.to_csv('data/doc_'+str(index+1)+'.tsv', sep='\t', index=False, header=False)

In [64]:
# Organizing the functions


    # list of unique words (terms) to be used to build the vocabulary
    words = []

    def cleanData(rawData):
        tokenizer = RegexpTokenizer(r'\w+')
        stop_words = set(stopwords.words("english"))

        # TODO translatoin
        #lang = detect(t4[0])
        #print(lang)
        #rawData = translator.translate(rawData, dest='en')

        # get words lowercased
        t0 = rawData.lower()
        # remove puctuations
        t1 = tokenizer.tokenize(t0)
        #t2Data = stemmer.stem(i for i in t1Data)
        #print(len(t1))
        # reomve stop words
        t2 =[]
        t2 = [t1[i] for i in range(0,len(t1)) if t1[i] not in stop_words]

        # stemm words
        t3 = [stemmer.stem(t2[i]) for i in range(0, len(t2))]

        # remove nummbers and strings starting with numbers
        t4 = [t3[i] for i in range(0, len(t3)) if t3[i][0].isdigit()==False]


        #print(t4)
        return(t4)

    def getWordsTSV(fid):
        fileWords=[]
        for i in open('data/doc_'+str(fid)+'.tsv', encoding="utf8"):
            #words.append(cleanData(i[0][4]))
            data = [i.strip('\n').split('\t') for i in open('data/doc_'+str(fid)+'.tsv',encoding="utf8")]
            for w in (cleanData(data[0][4].replace('\\n', ' '))):
                if w not in fileWords: fileWords.append(w)
        return(fileWords)


In [51]:
#print(*getWordsTSV(1))

In [65]:

for i in range(1,fileCount):
    t = getWordsTSV(i)
    for w in t:
        if w not in words:
            words.append(w)
print(words)

['welcom', 'stay', 'privat', 'room', 'queen', 'bed', 'detach', 'bathroom', 'second', 'floor', 'anoth', 'bedroom', 'sofa', 'avail', 'addit', 'guest', 'iah', 'airport', 'pick', 'drop', 'trip', 'stylish', 'fulli', 'remodel', 'home', 'upscal', 'nw', 'alamo', 'height', 'area', 'amaz', 'locat', 'hous', 'conveni', 'quiet', 'street', 'beauti', 'season', 'tree', 'prestigi', 'neighborhood', 'close', 'loop', 'town', 'featur', 'open', 'plan', 'origin', 'hardwood', 'full', 'independ', 'garden', 'tv', 'sleep', 'european', 'inspir', 'kitchen', 'top', 'line', 'decor', 'driveway', 'park', 'car', 'river', 'island', 'citi', 'well', 'maintain', 'san', 'jacinto', 'extra', 'temporari', 'visitor', 'cute', 'littl', 'situat', 'covet', 'acr', 'bryan', 'access', 'recent', 'purchas', 'mile', 'outsid', 'downtown', 'fort', 'worth', 'happi', 'restor', 'charact', 'built', 'minut', 'drive', 'offer', 'across', 'current', 'rejuven', 'gateway', 'place', 'lake', 'conro', 'famili', 'friend', 'activ', 'nightlif', 'love', 'o

#### PS
dunno why the stemmer is removing the last 'e' from any word
we need to look at the stem function again later

In [ ]:
#print(*words)
#words=[]

In [219]:
# To reset the words 
#words =[]

In [66]:
# writing the vocabulary file from the words 
wordsCount=0
with open('vocabulary.csv','wb') as vfile:
    for i in range(0,len(words)):
        vfile.write(str(wordsCount).encode())
        vfile.write(str('\t').encode())
        vfile.write(str(words[i]).encode())
        vfile.write('\n'.encode())
        wordsCount+=1

In [80]:
# creating index-file by loading vocabulary file and then comparing files with all vocabularys
with open('vocabulary.csv', newline='') as file:
    reader = csv.reader(file, delimiter='\t')
    voc = list(map(tuple, reader))
#print(voc)

index = {}
for i in range(len(voc)):
    tempL=[]

    for j in range(1,fileCount+1):
        temp = getWordsTSV(j)
        #print(temp)
        if voc[i][1] in temp:
            tempL.append(j)
            
        if len(tempL)!=0: index[i]= tempL
            
print(index)  

{0: [1, 5], 1: [1], 2: [1, 4, 10], 3: [1, 2, 3], 4: [1, 8], 5: [1, 8], 6: [1], 7: [1, 2, 4, 8], 8: [1], 9: [1, 2], 10: [1], 11: [1, 2, 4, 8, 9], 12: [1], 13: [1, 7, 8], 14: [1], 15: [1], 16: [1], 17: [1, 2, 7], 18: [1], 19: [1], 20: [1], 21: [2], 22: [2], 23: [2], 24: [2, 4, 5, 7], 25: [2], 26: [2], 27: [2], 28: [2], 29: [2], 30: [2], 31: [2], 32: [2, 3, 8], 33: [2], 34: [2], 35: [2, 5], 36: [2, 8], 37: [2], 38: [2], 39: [2], 40: [2, 4, 6], 41: [2, 3, 6], 42: [2], 43: [2], 44: [2], 45: [2], 46: [2], 47: [2, 5], 48: [2], 49: [2], 50: [2], 51: [2, 4], 52: [2], 53: [2], 54: [2], 55: [2], 56: [2], 57: [2], 58: [2], 59: [2], 60: [2], 61: [2, 5, 7], 62: [2], 63: [3], 64: [3], 65: [3], 66: [3], 67: [3], 68: [3], 69: [3], 70: [3], 71: [3], 72: [3], 73: [4], 74: [4], 75: [4], 76: [4], 77: [4, 7], 78: [4], 79: [4], 80: [5], 81: [5], 82: [5, 7, 8], 83: [5], 84: [5, 8], 85: [5], 86: [5], 87: [5], 88: [5], 89: [5], 90: [5], 91: [5], 92: [5], 93: [5, 7], 94: [5], 95: [5], 96: [5], 97: [5], 98: [6], 

In [346]:
#print(cleanData("德州农机大学(Texas A&amp;M University)北门附近2b1b其中的一间卧室提供日租。位置便利,步行进入校区只需要5分钟;室友为中国男生,好相处;日租时间段为2015年12月9日~30日。适合刚来学校没有车或者不打算买车的同学和老师(男性)"))

['德州农机大学', 'texa', 'amp', 'univers', '北门附近2b1b其中的一间卧室提供日租', '位置便利', '步行进入校区只需要5分钟', '室友为中国男生', '好相处', '日租时间段为2015年12月9日', '适合刚来学校没有车或者不打算买车的同学和老师', '男性']


In [67]:
def getID(term):
    with open('vocabulary.csv', newline='') as voc:
        line = csv.reader(voc, delimiter='\t')
        w = list(map(tuple, line))
        #print(w[0][1])
        for i in w:
            if term == i[1]:
                return (i[0])
        else:
            return None 

#getID('Hass')

In [68]:
# function takes term ID and returns the list of files containing the term
def getDocsWithID(tid):
    for k in index:
        if tid == k:
            return(index.get(k))
    return None

#getting error: TypeError: 'int' object is not iterable

In [497]:
getDocsWithID(5)

[1, 8]

In [69]:
# fixed it to check if the doc contains all terms
def getDocswithQuery(q):
    ts = cleanData(q)
    docsIDs = getDocsWithID(int(getID(ts[0])))
    if len(ts) == 0 : return None
    if len(ts)>1:
        docsIDs = getDocsWithID(int(getID(ts[0])))
        for t in ts:
            if getID(t) != None:
                tempDocs = getDocsWithID(int(getID(t)))
                if tempDocs != None:
                    docsIDs = set(docsIDs).intersection(tempDocs)
            return None
    return(list(docsIDs))

In [70]:
getDocsWithID(int(getID('bed')))

TypeError: 'int' object is not iterable

In [73]:
getDocswithQuery('bed private beautigul')

TypeError: 'int' object is not iterable

In [71]:
def getDetails(fid):
    with open('data/doc_'+str(fid)+'.tsv', encoding="utf8") as file:
        line = csv.reader(file, delimiter='\t')
        cls = list(map(str, *line))
        print(cls[7],cls[4], cls[2], cls[8])
        # TODO create a table and list the details


In [558]:
def getDetails(fid):
    detlist =[]
    with open('data/doc_'+str(fid)+'.tsv', encoding="utf8") as file:
        line = csv.reader(file, delimiter='\t')
        cls = list(map(str, *line))
        detlist= [cls[7],cls[4], cls[2], cls[8]]
        #TODO create a table and list the details
        return detlist

In [72]:

def printResults(query):
    docs = getDocswithQuery(query)
    #det =[['Title','Description', 'City','Url']]
    #for doc in docs:
    #    det.append(getDetails(doc))
    #columns = ('Title','Description', 'City','Url')
    #n_rows = len(det)
    html='<table align="left"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th></tr>'
    for doc in docs:
        l = getDetails(doc)
        html+='<tr><td>'+l[0]+'</td><td>'+l[1]+'</td><td>'+l[2]+'</td><td>'+l[3]+'</td></tr>'
    html+='</table>'
    display(HTML(html))
    #table = ff.create_table(det, height_constant=60)
    
    #plotly.offline.iplot(table, filename='simple_table')
    #plt.show()
    #return det[0][1]
    
    


In [75]:
printResults("Bed ")

2 Private rooms/bathroom 10min from IAH airport Welcome to stay in private room with queen bed and detached private bathroom on the second floor. Another private bedroom with sofa bed is available for additional guests. 10$ for an additional guest.\n10min from IAH airport\nAirport pick-up/drop off is available for $10/trip. Humble https://www.airbnb.com/rooms/18520444?location=Cleveland%2C%20TX


TypeError: 'NoneType' object is not subscriptable

In [76]:
printResults("bed")

2 Private rooms/bathroom 10min from IAH airport Welcome to stay in private room with queen bed and detached private bathroom on the second floor. Another private bedroom with sofa bed is available for additional guests. 10$ for an additional guest.\n10min from IAH airport\nAirport pick-up/drop off is available for $10/trip. Humble https://www.airbnb.com/rooms/18520444?location=Cleveland%2C%20TX


TypeError: 'NoneType' object is not subscriptable

In [564]:
print(getDocsWithID(160))

[9]


In [77]:
print(getDocswithQuery("a beautiful house with garden and beach"))

None


In [78]:
cleanData("Bed")

['bed']

In [79]:
print(getID('bed'))

5


In [ ]:
'''
from google.cloud import translate
translate_client = translate.Client()
text = u'Hello, world!'
target = 'ru'
# Translates some text into Russian
translation = translate_client.translate(
    text,
    target_language=target)

print(u'Text: {}'.format(text))
print(u'Translation: {}'.format(translation['translatedText']))
'''

{0: [1, 5], 1: [1], 2: [1, 4, 10], 3: [1, 2, 3], 4: [1, 8], 5: [1, 8], 6: [1], 7: [1, 2, 4, 8], 8: [1], 9: [1, 2], 10: [1], 11: [1, 2, 4, 8, 9], 12: [1], 13: [1, 7, 8], 14: [1], 15: [1], 16: [1], 17: [1, 2, 7], 18: [1], 19: [1], 20: [1], 21: [2], 22: [2], 23: [2], 24: [2, 4, 5, 7], 25: [2], 26: [2], 27: [2], 28: [2], 29: [2], 30: [2], 31: [2], 32: [2, 3, 8], 33: [2], 34: [2], 35: [2, 5], 36: [2, 8], 37: [2], 38: [2], 39: [2], 40: [2, 4, 6], 41: [2, 3, 6], 42: [2], 43: [2], 44: [2], 45: [2], 46: [2], 47: [2, 5], 48: [2], 49: [2], 50: [2], 51: [2, 4], 52: [2], 53: [2], 54: [2], 55: [2], 56: [2], 57: [2], 58: [2], 59: [2], 60: [2], 61: [2, 5, 7], 62: [2], 63: [3], 64: [3], 65: [3], 66: [3], 67: [3], 68: [3], 69: [3], 70: [3], 71: [3], 72: [3], 73: [4], 74: [4], 75: [4], 76: [4], 77: [4, 7], 78: [4], 79: [4], 80: [5], 81: [5], 82: [5, 7, 8], 83: [5], 84: [5, 8], 85: [5], 86: [5], 87: [5], 88: [5], 89: [5], 90: [5], 91: [5], 92: [5], 93: [5, 7], 94: [5], 95: [5], 96: [5], 97: [5], 98: [6], 

In [566]:
q = input()
q_clean = cleanData(q)
for w in q_clean

SyntaxError: invalid syntax (<ipython-input-566-8de38025c9f0>, line 3)

In [570]:
output = "<html><body><table>"
for key in q_clean:
  output += "<tr><td>%s</td><td>{}</td></tr>".format(key)
output += "</table></body></html>"
print(output)



<html><body><table><tr><td>%s</td><td>need</td></tr><tr><td>%s</td><td>beati</td></tr><tr><td>%s</td><td>appart</td></tr></table></body></html>


In [582]:
from IPython.display import HTML, display
data = [[1,2,3],
         [4,5,6],
         [7,8,9],
         ]
out='<table><th><td><b>Term<b></td></th>'
for k in q_clean:
    out +='<tr><td>'+str(k)+'</td></tr>'
out+='</table>'

display(HTML(out))

need
beati
appart


In [84]:
#create dictionary with term frequency per fid by using the modified getWordsTSV function below
#obsolete now as tf for every word is directly calculated in getTFIDF
'''
def getTF(fid):
    tfd = {}
    words = getWordsTSV_dup(fid)
    for word in words:
        if word in tfd:
            tfd[word] =  tfd[word]+1
            #replace word with term id from vocabulary
        else:
            tfd[word] = 1
    for tfs in tfd:
        tfd[tfs] = tfd[tfs]/len(words)
    return tfd
'''

#modified getWordsTSV function that creates a list with all words in the doc and does not remove the duplicates
def getWordsTSV_dup(fid):
    fileWords=[]
    for i in open('data/doc_'+str(fid)+'.tsv', encoding="utf8"):
        #words.append(cleanData(i[0][4]))
        data = [i.strip('\n').split('\t') for i in open('data/doc_'+str(fid)+'.tsv',encoding="utf8")]
        for w in (cleanData(data[0][4].replace('\\n', ' '))):
            fileWords.append(w)
    return(fileWords)
    

In [189]:
#calculate the inverse data frquency(idf) of a word
def getIDF(word):
    '''#set counter for occurences of the term in the documents
    #that contain the term to 0
    #N = 0'''
    #get the ids of the documents in which the term occures
    files = getDocsWithID(int(getID(word)))
    
    ''' 
    #iterate over the list with the document ids
    for file in files:
        #get the words of each document
        words = getWordsTSV_dup(file)
        #check how often the term occurs in the documents
        for x in words:
            if x == word: 
                N += 1+
                '''
    #number of docs in which the terms occurs it the length of files
    #filecount = total number of files
    idf = math.log(fileCount/len(files))
    return idf
    
    

In [186]:
#calculate the tfidf for one word
def getTFIDF(word, docID):
    
    files = getDocsWithID((int(getID(word))))
    print(files)
    words =[]
    N = 0
    for file in files:
        words.extend(getWordsTSV_dup(file))
    for x in words:
        #print(x)
        if x == word:
             N += 1
    tf = N/len(words)
    tfidf = tf * getIDF(word)
    print(word)
    print(words)
    print(N)
    print(tf)
    print(tfidf)

    
    '''       
        tfd = {}
    words = getWordsTSV_dup(fid)
    for word in words:
        if word in tfd:
            tfd[word] =  tfd[word]+1
            #replace word with term id from vocabulary
        else:
            tfd[word] = 1
    for tfs in tfd:
        tfd[tfs] = tfd[tfs]/len(words)
    '''

In [187]:
getTFIDF('bed')

[1, 8]
bed
['welcom', 'stay', 'privat', 'room', 'queen', 'bed', 'detach', 'privat', 'bathroom', 'second', 'floor', 'anoth', 'privat', 'bedroom', 'sofa', 'bed', 'avail', 'addit', 'guest', 'addit', 'guest', 'iah', 'airport', 'airport', 'pick', 'drop', 'avail', 'trip', 'beauti', 'bedroom', 'queen', 'size', 'bed', 'closet', 'pet', 'hous', 'alway', 'clean', 'bathroom', 'share', 'suppli', 'towel', 'shampoo', 'avail', 'mile', 'downtown', 'tcu', 'tcc', 'stockyard']
3
0.061224489795918366
0.024824394373969248


In [171]:
getWordsTSV_dup(8)

['beauti',
 'bedroom',
 'queen',
 'size',
 'bed',
 'closet',
 'pet',
 'hous',
 'alway',
 'clean',
 'bathroom',
 'share',
 'suppli',
 'towel',
 'shampoo',
 'avail',
 'mile',
 'downtown',
 'tcu',
 'tcc',
 'stockyard']

In [134]:
getWordsTSV_dup(5)

['welcom',
 'origin',
 'home',
 'recent',
 'purchas',
 'home',
 'mile',
 'outsid',
 'downtown',
 'fort',
 'worth',
 'happi',
 'restor',
 'origin',
 'charact',
 'home',
 'built',
 'minut',
 'drive',
 'downtown',
 'offer',
 'across',
 'street',
 'current',
 'rejuven',
 'gateway',
 'park']